# Sentiment Analysis using BERT

## Installation 

In [1]:
!pip install torch torchvision torchaudio

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

**Import Libraries**: Start by importing the necessary libraries and modules.

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import warnings

warnings.filterwarnings("ignore", message="torch.utils._pytree._register_pytree_node is deprecated")


**Load Pre-trained Model**: Load the pre-trained BERT model and tokenizer.

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

**Encode Text and Analyze Sentiment**: Encode the text you want to analyze using the tokenizer, pass the encoded tokens through the BERT model to get the sentiment logits, and determine the sentiment score based on the maximum logit value.

In [5]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [6]:
result = model(tokens)

In [7]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [8]:
int(torch.argmax(result.logits))+1

4

**Collect and Analyze Reviews**: Collect reviews from a website (e.g., Yelp) using web scraping techniques, perform sentiment analysis on the collected reviews, and store the results in a DataFrame.

In [9]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [10]:
reviews

["Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. 

In [11]:
import numpy as np
import pandas as pd

In [12]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [13]:
df['review'].iloc[0]

"Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!"

In [14]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [15]:
sentiment_score(df['review'].iloc[1])

4

In [16]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [17]:
df

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,4
1,Six of us met here for breakfast before our wa...,4
2,Great place with delicious food and friendly s...,5
3,"Great service, lovely location, and really ama...",5
4,Great food amazing coffee and tea. Short walk ...,5
5,Ricotta hot cakes! These were so yummy. I ate ...,5
6,It was ok. Had coffee with my friends. I'm new...,3
7,We came for brunch twice in our week-long visi...,4
8,I came to Social brew cafe for brunch while ex...,5
9,Great staff and food. Must try is the pan fri...,5


In [18]:
df['review'].iloc[3]

"Great service, lovely location, and really amazing food. Words don't do justice. We had the mushroom parm bruschetta and the loaded double double. Both were so tasty. Also love the Aussie black tea and a flat white. Wish I had more mornings in Sydney to eat breakfast there. Highly recommend."